# KODLAR

In [120]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [102]:
r = requests.get('https://www.sabah.com.tr/yazarlar/arsiv')

soup = BeautifulSoup(r.content, "html.parser")

figcaption = soup.section.find_all('figcaption')

authorLinks = []
authors = []

for ahref in figcaption:
    authors.append(ahref.get_text())
    authorLinks.append('https://www.sabah.com.tr' + ahref.a.get('href'))

In [109]:
a_cp = {}
for i in range(len(authors)):
    try:
        r = requests.get(authorLinks[i])
    except (requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as err:
        i -= 1
        continue
    else:
        soup = BeautifulSoup(r.content, "html.parser")

        div = soup.section.find('div', {'class' : 'row writerReadList custom04 yazar-infinite-load'})
        if "None" in str(div):
            i -= 1
            continue
        figcaption = div.find_all('figcaption')

        cp = []

        for ahref in figcaption:
            cp.append('https://www.sabah.com.tr' + ahref.a.get('href'))
        
        a_cp[authors[i]] = cp

In [128]:
data = []
for author in a_cp:
    for i in range(len(a_cp[author])):
        try:
            r = requests.get(a_cp[author][i])
        except (requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as err:
            i -= 1
            continue
        else:
            soup = BeautifulSoup(r.content, "html.parser")

            result = soup.find_all('div', attrs={'class':'col-sm-12'})
            
            if result[0].find('span', attrs={'class': 'postInfo'}) is None:
                info = np.nan
            else:
                info =result[0].find('span', attrs={'class': 'postInfo'}).get_text()
            
            if result[0].find('span', attrs={'class': 'postInfo'}) is None:
                title = np.nan
            else:
                title = result[0].find('strong', attrs={'class': 'postTitle'}).get_text()
            
            if result[0].find('span', attrs={'class': 'postInfo'}) is None:
                caption = np.nan
            else:
                caption = result[0].find('h1', attrs={'class': 'postCaption'}).get_text()
            
            text = result[0].find('div', attrs={'class': 'newsBox'})
            txt = text.get_text()
            txt = txt.split("\n")

            for j in range(0, len(txt) - 2):
                if "Bu köşe yazısını aşağıdaki linke tıklayarak sesli bir şekilde dinleyebilirsiniz" in txt[j]:
                    del txt[j]
                elif  " | " in txt[j]:
                    del txt[j]
                
            text = ''
            for j in range(len(txt)):
                text += "\n" + txt[j] 
            
            data.append((info, title, caption, a_cp[author][i], text))
df = pd.DataFrame(data = data, columns = ['info', 'title', 'caption', 'link', 'text'])
newData = df.to_csv('newData.csv', index = True)
df

,info,title,caption,link,text
0,"21 Şubat 2021, Pazar",MEHMET BARLAS,Dostlar da dost olmayanlar da Türkiye’nin neyi...,https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nCumhurbaşkanı Erdoğan'ın yönetimindeki Tür...
1,"20 Şubat 2021, Cumartesi",MEHMET BARLAS,Muhalefetsiz demokrasi olmaz ama bu çeşit muha...,https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nSizler de merak etmiyor musunuz? CHP Genel...
2,"19 Şubat 2021, Cuma",MEHMET BARLAS,Her iki Türk’ten biri yurt dışına göç etmek mi...,https://www.sabah.com.tr/yazarlar/barlas/2021/...,"\n\nAlman gazetelerinden ""Der Tagesspiegel"", T..."
3,"18 Şubat 2021, Perşembe",MEHMET BARLAS,"Cumhurbaşkanı, demokrasi tarihinin en ağır suç...",https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nÇok partili demokrasiye geçtiğimiz günden ...
4,"17 Şubat 2021, Çarşamba",MEHMET BARLAS,"Artık gündemin ana maddesi, terörle sonuna kad...",https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nBelli ki Türkiye'nin ana gündemi artık ter...
...,...,...,...,...,...
2452,"11 Şubat 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Önemli gün,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nBugün Antalya için çok önemli bir gün.\r\n...
2453,"04 Şubat 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Acılar üstüne,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nAntalyamız bu hafta acılar yaşadı.\r\nAynı...
2454,"28 Ocak 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Haydi maça,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nAntalya'da spor ve kardeşlik adına önemli ...
2455,"21 Ocak 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Laf-ı güzaf,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nDün bizim gazetenin manşet haberi Antalya ...


In [126]:
for i in data:
    with open(f'metin/{str(i)}.txt', 'w', encoding = 'utf-8') as f:
        f.write(data)

FileNotFoundError: [Errno 2] No such file or directory: 'metin/(\'21 Şubat 2021, Pazar\', \'MEHMET BARLAS\', \'Dostlar da dost olmayanlar da Türkiye’nin neyi hedeflediğini merak ediyor\', \'https://www.sabah.com.tr/yazarlar/barlas/2021/02/21/dostlar-da-dost-olmayanlar-da-turkiyenin-neyi-hedefledigini-merak-ediyor\', \'\\n\\nAntalya güzellikleri ile dünyaya nam salan cennet kentimiz.\\r\\nAtatürk\\\'ün dediği gibi "Şüphesiz ki Antalya dünyanın en güzel yeridir." Böylesine güzel bir şehrimiz olan Antalyamızda iyi gelişmelerin yanında, kötü ve çirkin gelişmeler de yaşanmıyor değil.\\r\\nİnanın yaşanan kütü ve çirkin gelişmeler ziyadesiyle bizleri üzüyor ve mahcup ediyor.\\r\\nBugün sizlerle öncelikle iyi gelişmeleri ve yapılanları paylaşmak istiyorum.\\r\\nAkdeniz Üniversitesi Kampüsü\\\'nde iki caddeye 15 Temmuz darbe girişimi sırasında şehit düşen hukuk fakültesi öğrencisi Yasin Naci Ağaroğlu ve polis memuru Muhammet Oğuz Kılınç\\\'ın adının verilmesi.\\r\\nAkdeniz Üniversitesi Rektörü 

In [118]:
data

[('21 Şubat 2021, Pazar',
  'MEHMET BARLAS',
  'Dostlar da dost olmayanlar da Türkiye’nin neyi hedeflediğini merak ediyor',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/21/dostlar-da-dost-olmayanlar-da-turkiyenin-neyi-hedefledigini-merak-ediyor',
  '\n\nAntalya güzellikleri ile dünyaya nam salan cennet kentimiz.\r\nAtatürk\'ün dediği gibi "Şüphesiz ki Antalya dünyanın en güzel yeridir." Böylesine güzel bir şehrimiz olan Antalyamızda iyi gelişmelerin yanında, kötü ve çirkin gelişmeler de yaşanmıyor değil.\r\nİnanın yaşanan kütü ve çirkin gelişmeler ziyadesiyle bizleri üzüyor ve mahcup ediyor.\r\nBugün sizlerle öncelikle iyi gelişmeleri ve yapılanları paylaşmak istiyorum.\r\nAkdeniz Üniversitesi Kampüsü\'nde iki caddeye 15 Temmuz darbe girişimi sırasında şehit düşen hukuk fakültesi öğrencisi Yasin Naci Ağaroğlu ve polis memuru Muhammet Oğuz Kılınç\'ın adının verilmesi.\r\nAkdeniz Üniversitesi Rektörü Prof. Dr. Mustafa Ünal başkanlığında toplanan Üniversite Senatosu aldığı kararla 

# DENEME

In [65]:
deneme = soup.find_all('div', attrs={'class':'col-sm-12'})
deneme

[<div class="col-md-8 col-sm-12 column-left">
 <div class="row frame infinite" data-index="1">
 <div class="col-sm-12">
 <script type="application/ld+json">
         {
         "@context": "https://schema.org",
         "@type": "BreadcrumbList",
         "itemListElement": [
             
                 {
                 "@type": "ListItem",
                 "position": 1,
                 "name": "Haberler", 
                 "item": "https://www.sabah.com.tr/" 
                 }
             
                 ,
             
                 {
                 "@type": "ListItem",
                 "position": 2,
                 "name": "Ömer Durna (akdeniz)", 
                 "item": "https://www.sabah.com.tr/yazarlar/omer-durna-akdeniz/arsiv?getall=true" 
                 }
             
         ]
         }
     </script>
 <div class="breadcrumb">
 <span><a href="https://www.sabah.com.tr/">Haberler</a></span>
 <span><a href="https://www.sabah.com.tr/yazarlar/omer-durna-akde

In [89]:
info = deneme[0].find('span', attrs={'class': 'postInfo'})
title = deneme[0].find('strong', attrs={'class': 'postTitle'})
caption = deneme[0].find('h1', attrs={'class': 'postCaption'})
text = deneme[0].find('div', attrs={'class': 'newsBox'})
print(info.get_text(), title.get_text(), caption.get_text(), text.get_text())

14 Ocak 2017, Cumartesi ÖMER DURNA (AKDENİZ) İyi, kötü, çirkin 
Antalya güzellikleri ile dünyaya nam salan cennet kentimiz.
Atatürk'ün dediği gibi "Şüphesiz ki Antalya dünyanın en güzel yeridir." Böylesine güzel bir şehrimiz olan Antalyamızda iyi gelişmelerin yanında, kötü ve çirkin gelişmeler de yaşanmıyor değil.
İnanın yaşanan kütü ve çirkin gelişmeler ziyadesiyle bizleri üzüyor ve mahcup ediyor.
Bugün sizlerle öncelikle iyi gelişmeleri ve yapılanları paylaşmak istiyorum.
Akdeniz Üniversitesi Kampüsü'nde iki caddeye 15 Temmuz darbe girişimi sırasında şehit düşen hukuk fakültesi öğrencisi Yasin Naci Ağaroğlu ve polis memuru Muhammet Oğuz Kılınç'ın adının verilmesi.
Akdeniz Üniversitesi Rektörü Prof. Dr. Mustafa Ünal başkanlığında toplanan Üniversite Senatosu aldığı kararla ahde vefa örneği sergiledi.
Bundan sonra kampüs içindeki Doğu-Batı Caddesi Yasin Naci Ağaroğlu'nun, Güney-Kuzey Caddesi ise Muhammet Oğuz Kılınç'ın adı ile anılacak.
Üniversitemiz bu iki şehidimizin adını ölümsüzleş

{'MEHMET BARLAS': ['https://www.sabah.com.tr/yazarlar/barlas/2021/02/21/dostlar-da-dost-olmayanlar-da-turkiyenin-neyi-hedefledigini-merak-ediyor',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/20/muhalefetsiz-demokrasi-olmaz-ama-bu-cesit-muhalefet-de-demokrasiyi-hasta-etmez-mi',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/19/her-iki-turkten-biri-yurt-disina-goc-etmek-mi-istiyormus',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/18/cumhurbaskani-demokrasi-tarihinin-en-agir-suclamalarini-seslendirdi',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/17/artik-gundemin-ana-maddesi-terorle-sonuna-kadar-mucadeledir',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/16/cumhurbaskani-erdogan-pkk-terorunun-yaninda-yer-alan-abdyi-cok-sert-uyardi',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/15/bir-istanbullu-olarak-kadir-topbasa-tesekkur-borcluyuz',
  'https://www.sabah.com.tr/yazarlar/barlas/2021/02/14/trump-ve-erdogan-takintili-biden-amerikasi-ile-nasil-basa',

In [70]:
r = requests.get('https://www.sabah.com.tr/yazarlar/barlas/arsiv?getall=true')

In [337]:
soup = BeautifulSoup(r.content, "html.parser")

asd = soup.section.find('div', {'class' : 'row writerReadList custom04 yazar-infinite-load'})
figcaption = asd.find_all('figcaption')

cp = []

for ahref in figcaption:
    cp.append('https://www.sabah.com.tr' + ahref.a.get('href'))

In [32]:
len(metin)

2431

In [332]:
r = requests.get('https://www.sabah.com.tr/yazarlar/muderrisoglu/2021/02/02/turk-ozel-sektorunun-borcu')

In [333]:
soup = BeautifulSoup(r.content, "html.parser")

txt = soup.section.find('div', {'class' : 'newsBox'})
txt = txt.get_text()
txt = txt.split("\n")


for i in range(0, len(txt) - 2):
    if "Bu köşe yazısını aşağıdaki linke tıklayarak sesli bir şekilde dinleyebilirsiniz" in txt[i]:
        del txt[i]
    elif  " | " in txt[i]:
        del txt[i]
text = ''
for i in range(len(txt)):
    text += "\n" + txt[i]        


In [334]:
text 

'\n\nDikkatli gözler, meşru muhalefet davranışı dışında siyasal alanı da aşan, toplumun sinir uçlarına basan organize faaliyetlerin giderek sıklaştığını fark ediyorlar. Özellikle de sosyal medya üzerinden düzenlenen kampanyalar, arkasındakiler ve kullandıkları nefret dili hafife alınacak cinsten değil. Kimi zaman üniversite gençliği, kimi zaman kırılgan konumdaki ekonomik kesimlerin hassasiyeti manipüle edilebiliyor. Bir damar yakalandığı andan itibaren, süreç dalga dalga büyütülebiliyor! Türkiye\'nin aşmaya çalıştığı sorunları psikolojik harekat unsuru olarak kullanabilen odaklara karşı, klasik yöntemlerin yanı sıra proaktif politikalar da gerekli görünüyor.\r\n\r\n***\n\n\n \n\n \nBu tespitten sonra farklı bir konuya değinmek istiyorum. Konumuz, özel sektör ile hükümet ilişkileri bağlamındaki bir hususla ilgili. Zira meseleye bir başka pencereden de bakılması taraftarıyım.\r\nBilindiği üzere Cumhurbaşkanı Tayyip Erdoğan, ekonomik büyümenin merkezine Türk özel sektörünü yerleştirmiş b